In [2]:
import pandas as pd

In [3]:
#df = pd.read_csv("../hawkdove_2023-09-25T150643_380857.csv")
# more recent run that includes data collection for agent cumulative points
# df = pd.read_csv("../hawkdove_risk-single_2023-09-27T154226_942256.csv")
# smaller version - only 200 rounds
df = pd.read_csv("../hawkdove_risk-single_2023-09-27T175109_555307.csv")


In [4]:
df.head()

,RunId,iteration,Step,grid_size,risk_attitudes,agent_risk_level,max_agent_points,percent_hawk,AgentID,risk_level,choice,points
0,0,0,0,20,single,0,21.0,0.5625,NaN,NaN,NaN,NaN
1,0,0,1,20,single,0,30.0,0.7600,0.0,0.0,dove,13.5
2,0,0,1,20,single,0,30.0,0.7600,1.0,0.0,dove,11.3
3,0,0,1,20,single,0,30.0,0.7600,2.0,0.0,dove,13.5
4,0,0,1,20,single,0,30.0,0.7600,3.0,0.0,hawk,6.0


In [5]:
model_df = df[df.AgentID.isna()]
model_df.head()

,RunId,iteration,Step,grid_size,risk_attitudes,agent_risk_level,max_agent_points,percent_hawk,AgentID,risk_level,choice,points
0,0,0,0,20,single,0,21.0,0.5625,NaN,NaN,NaN,NaN
80001,1,0,0,20,single,1,21.0,0.5150,NaN,NaN,NaN,NaN
160002,2,0,0,20,single,2,24.0,0.4800,NaN,NaN,NaN,NaN
240003,3,0,0,20,single,3,24.0,0.4575,NaN,NaN,NaN,NaN
320004,4,0,0,20,single,4,24.0,0.5300,NaN,NaN,NaN,NaN


In [6]:
# get model-level data across all rounds and runs
run_df = df[['RunId', 'iteration', 'Step', 'agent_risk_level', 'percent_hawk']]

In [7]:
run_df = run_df.drop_duplicates()
run_df

,RunId,iteration,Step,agent_risk_level,percent_hawk
0,0,0,0,0,0.5625
1,0,0,1,0,0.7600
401,0,0,2,0,0.9850
801,0,0,3,0,0.2050
1201,0,0,4,0,0.7600
...,...,...,...,...,...
718009,8,0,196,8,0.0000
718409,8,0,197,8,0.0000
718809,8,0,198,8,0.0000
719209,8,0,199,8,0.0000


In [8]:
import altair as alt

alt.data_transformers.disable_max_rows()

# alt.Chart(run_df[run_df.Step < 100]).mark_line().encode(
alt.Chart(run_df).mark_line().encode(    
    x='Step',
    y='percent_hawk',
    color='agent_risk_level:N',
).properties(
    width=800,
    height=300
)

alt.Chart(...)

In [9]:
runzero = run_df[run_df.RunId == 0]
runzero

,RunId,iteration,Step,agent_risk_level,percent_hawk
0,0,0,0,0,0.5625
1,0,0,1,0,0.7600
401,0,0,2,0,0.9850
801,0,0,3,0,0.2050
1201,0,0,4,0,0.7600
...,...,...,...,...,...
78001,0,0,196,0,0.7600
78401,0,0,197,0,0.9850
78801,0,0,198,0,0.2050
79201,0,0,199,0,0.7600


In [10]:
run_one = run_df[run_df.RunId == 1]


In [11]:
run_zero_chart = alt.Chart(runzero[runzero.Step < 150]).mark_line().encode(
    x='Step', # alt.X('Step', scale=alt.Scale(domain=[0, 1])),
    y='percent_hawk',
    # color='agent_risk_level:N',
).properties(
    width=800,
    height=300
)
run_zero_chart

alt.Chart(...)

In [12]:
# how to work with this oscillating pattern of alternating hawks?
# can we use a rolling mean?

line = alt.Chart(runzero).mark_line(
    color='red',
    size=3
).transform_window(
    rolling_mean='mean(percent_hawk)',
    frame=[-15, 15]
).encode(
    x='Step',
    y='rolling_mean:Q'
).properties(
    width=800,
    height=300
)

points = alt.Chart(runzero).mark_line().encode(
    x='Step',
    y='percent_hawk'
)

points + line


alt.LayerChart(...)

In [13]:
# create and display charts for each run / risk level

charts = []

for i in range(8):
    run_i = run_df[run_df.RunId == i]
    risk_level = run_i.agent_risk_level.unique()[0]
    run_chart = alt.Chart(run_i).mark_line().encode(
        x='Step',
        y=alt.Y('percent_hawk', scale=alt.Scale(domain=[0, 1.0]))
        # color='agent_risk_level:N',
    ).properties(
        # title=f'Run {i}, risk level {risk_level}',
        title=f'Risk level {risk_level}',
        width=600,
        height=90
    )
    charts.append(run_chart)

combined_chart = None
for c in charts:
    if combined_chart is None:
        combined_chart = c
    else:
        combined_chart = alt.vconcat(combined_chart, c)

combined_chart

alt.VConcatChart(...)

In [14]:
# do the same thing, but display beginning instead of end and add the rolling mean

rollmean_charts = []

for i in range(8):
    run_i = run_df[run_df.RunId == i]
    risk_level = run_i.agent_risk_level.unique()[0]
    run_chart = alt.Chart(run_i).mark_line().encode(
        x='Step',
        y=alt.Y('percent_hawk', scale=alt.Scale(domain=[0, 1.0]))
        # color='agent_risk_level:N',
    ).properties(
        # title=f'Run {i}, risk level {risk_level}',
        title=f'Risk level {risk_level}',
        width=800,
        height=90
    )
    rollmean_line = alt.Chart(runzero[runzero.Step < 300]).mark_line(
        color='red',
        size=3
    ).transform_window(
        rolling_mean='mean(percent_hawk)',
        frame=[-15, 15]
    ).encode(
        x='Step',
        y='rolling_mean:Q'
    )
    
    rollmean_charts.append(run_chart + rollmean_line)

rollmean_combined_chart = None
for c in rollmean_charts:
    if rollmean_combined_chart is None:
        rollmean_combined_chart = c
    else:
        rollmean_combined_chart = alt.vconcat(rollmean_combined_chart, c)

rollmean_combined_chart

alt.VConcatChart(...)

## percent hawk by risk level

In [15]:
# for each run (= risk level), what are upper and lower values and rolling mean for % hawk?

hawkstats = []

for i in range(8) :
    # get the end of the run; we stopped at 1000 but it stabilized very early; use last 50 rounds
    # in this run, we only ran for 200 iterations, since it stabilizes quickly
    run_i = run_df[run_df.RunId == i]
    phawk_vals = run_i.percent_hawk.describe()
    # add one entry for each value with a type, so we can graph all at once in altair with a legend
    hawkstats.append({
        'risk_level': i, 
        'max': run_i.percent_hawk.max(), 
        'mean': run_i.percent_hawk.mean(), 
        'min': run_i.percent_hawk.min()
    })

hawkstats_df = pd.DataFrame(hawkstats)
hawkstats_df

,risk_level,max,mean,min
0,0,0.9850,0.651779,0.2050
1,1,0.9900,0.589080,0.2200
2,2,1.0000,0.551045,0.2825
3,3,0.9725,0.514490,0.1700
4,4,0.8175,0.485348,0.0200
5,5,0.6800,0.442363,0.0125
6,6,0.7850,0.410149,0.0025
7,7,0.8225,0.354192,0.0025


In [16]:
# for each run (= risk level), what are upper and lower values and rolling mean for % hawk?
# format in a way we can easily graph together with altair

alt_hawkstats = []

for i in range(8) :
    # get the end of the run; we stopped at 1000 but it stabilized very early; use last 50 rounds
    # ran for 200 rounds; omit first 50 before it stabilized
    run_i = run_df[(run_df.RunId == i) & (run_df.Step > 50)]
    phawk_vals = run_i.percent_hawk.describe()
    # add one entry for each value with a type, so we can graph all at once in altair with a legend
    alt_hawkstats.extend([
        {'risk_level': i, 'value': run_i.percent_hawk.max(), 'type': 'max'},
        {'risk_level': i, 'value': run_i.percent_hawk.mean(), 'type': 'mean'},
        {'risk_level': i, 'value': run_i.percent_hawk.min(), 'type': 'min'},
    ])

alt_hawkstats_df = pd.DataFrame(alt_hawkstats)
alt_hawkstats_df

,risk_level,value,type
0,0,0.985000,max
1,0,0.650000,mean
2,0,0.205000,min
3,1,0.990000,max
4,1,0.587500,mean
5,1,0.222500,min
6,2,1.000000,max
7,2,0.550000,mean
8,2,0.282500,min
9,3,0.972500,max


In [17]:
alt.Chart(alt_hawkstats_df).mark_line().encode(
    x='risk_level:N', 
    y='value',
    color=alt.Color('type').scale(domain=['min', 'mean', 'max'], range=['purple', 'blue', 'orange'])
).properties(
    title='% hawk by risk level (min, max, mean)',
    width=500,
    height=400
)

alt.Chart(...)

## points by risk level

In [18]:
# what about points?

# get points at the last round only, so we're looking at the end state

last_round_n = df.Step.max()

last_round = df[df.Step == last_round_n]

points_mean = last_round.groupby('RunId', as_index=False).aggregate('points').mean() # : ['mean', 'sum']})
points_mean

,RunId,points
0,0,1527.5400
1,1,1797.3435
2,2,1956.5375
3,3,2043.0240
4,4,2150.2400
5,5,2288.3545
6,6,2417.1090
7,7,2492.1570
8,8,3355.4040


In [19]:

alt.Chart(points_mean).mark_bar().encode(
    x=alt.Y('RunId:N', title='risk level'),
    y=alt.Y('points', title='average points'),
).properties(
    title='average points by risk level',
    width=500,
    height=400
)

alt.Chart(...)

In [20]:
# what about min/max?

# aggregrate separately so we can graph together in altair
points_mean['type'] = 'mean'

points_min = last_round.groupby('RunId', as_index=False).aggregate('points').min()
points_min['type'] = 'min'

points_max = last_round.groupby('RunId', as_index=False).aggregate('points').max()
points_max['type'] = 'max'

points_combined = pd.concat([points_mean, points_min, points_max])

points_combined

,RunId,points,type
0,0,1527.5400,mean
1,1,1797.3435,mean
2,2,1956.5375,mean
3,3,2043.0240,mean
4,4,2150.2400,mean
5,5,2288.3545,mean
6,6,2417.1090,mean
7,7,2492.1570,mean
8,8,3355.4040,mean
0,0,975.6000,min


In [21]:
alt.Chart(points_combined).mark_line().encode(
    x=alt.Y('RunId:N', title='risk level'),
    y=alt.Y('points', title='average points'),
    color='type'
).properties(
    title='points by risk level',
    width=500,
    height=400
)

alt.Chart(...)

In [22]:
last_round

,RunId,iteration,Step,grid_size,risk_attitudes,agent_risk_level,max_agent_points,percent_hawk,AgentID,risk_level,choice,points
79601,0,0,200,20,single,0,2285.4,0.985,0.0,0.0,dove,1658.3
79602,0,0,200,20,single,0,2285.4,0.985,1.0,0.0,dove,1656.1
79603,0,0,200,20,single,0,2285.4,0.985,2.0,0.0,dove,1585.7
79604,0,0,200,20,single,0,2285.4,0.985,3.0,0.0,dove,1679.4
79605,0,0,200,20,single,0,2285.4,0.985,4.0,0.0,dove,1909.5
...,...,...,...,...,...,...,...,...,...,...,...,...
720004,8,0,200,20,single,8,3381.0,0.000,395.0,8.0,dove,3355.2
720005,8,0,200,20,single,8,3381.0,0.000,396.0,8.0,dove,3352.2
720006,8,0,200,20,single,8,3381.0,0.000,397.0,8.0,dove,3355.2
720007,8,0,200,20,single,8,3381.0,0.000,398.0,8.0,dove,3355.2


In [30]:
alt.Chart(last_round).mark_boxplot(extent="min-max").encode(
    alt.X("points:Q").scale(zero=False),
    alt.Y("agent_risk_level:N", title="risk level"),
).properties(
    title='range of points per agent by risk level',
    width=500,
    height=400
)


alt.Chart(...)

In [34]:
# for each run (= risk level), what are upper and lower values for individual points?

points = []

for i in range(8) :
    run_i = last_round[last_round.RunId == i]
    # add one entry for each value with a type, so we can graph all at once in altair with a legend
    points.append({
        'risk_level': i, 
        'max': run_i.points.max(), 
        'mean': run_i.points.mean(), 
        'min': run_i.points.min()
    })

points_df = pd.DataFrame(points)
points_df

,risk_level,max,mean,min
0,0,2285.4,1527.5400,975.6
1,1,2411.0,1797.3435,1132.5
2,2,2574.6,1956.5375,1454.4
3,3,2657.7,2043.0240,1658.3
4,4,2697.3,2150.2400,1809.0
5,5,2768.4,2288.3545,1810.8
6,6,2926.6,2417.1090,1954.9
7,7,3723.6,2492.1570,1956.0
